# 使用DOCPLEX求解线性规划问题

## 导入docplex库

In [28]:
from docplex.mp.model import Model

## 列出线性规划问题的矩阵表达形式

In [29]:
num_variables = 3
MaxMin = "max"
A = [[1, 1, 1],
     [9, 6, 8],
     [12, 16, 13]]
b = [200, 1566, 2880]
C = [350, 300, 320]

## 创建线性规划模型

In [30]:
model = Model(name='linear_programming_example')
x = model.continuous_var_list(num_variables, name='x')

## 将约束条件添加到模型中

In [31]:
for i in range(len(A)):
    model.add_constraint(sum(A[i][j] * x[j] for j in range(num_variables)) <= b[i])

## 定义目标函数

In [32]:
if MaxMin == "max":
    model.maximize(sum(x * y for x, y in zip(C, model.iter_continuous_vars())))
elif MaxMin == "min":
    model.minimize(sum(x * y for x, y in zip(C, model.iter_continuous_vars())))
else:
    model.maximize(sum(x * y for x, y in zip(C, model.iter_continuous_vars())))

## 求解线性规划问题

In [33]:
solution = model.solve()

## 打印最优解

In [34]:
print("The optimal solution is:")
for i in range(num_variables):
    print(f"\tx{i + 1} = {solution.get_value(x[i])}")
print("\tObjective value =", solution.get_objective_value())

The optimal solution is:
	x1 = 122.0
	x2 = 78.0
	x3 = 0
	Objective value = 66100.0


## 获取cplex对象

In [35]:
cpx = model.get_engine().get_cplex()

## 打印灵敏度分析的报告

In [36]:
print("Sensitivity Analysis:")
print("\tOBJ Sensitivity Ranges")
print(cpx.solution.sensitivity.objective())
print(cpx.solution.sensitivity.lower_bounds())
print(cpx.solution.sensitivity.upper_bounds())
print(cpx.solution.sensitivity.bounds())
print(cpx.solution.sensitivity.rhs())
# 打印对偶解
print("The dual solution is:")
print("Dual values of the constraints:")
for c in model.iter_constraints():
    print("{}: {:.2f}".format(c.name, c.dual_value))
print("Reduced costs of the variables:")
for v in model.iter_variables():
    print("{}: {:.2f}".format(v.name, v.reduced_cost))

Sensitivity Analysis:
	OBJ Sensitivity Ranges
[(330.0, 450.0), (260.00000000000006, 350.0), (-1e+20, 333.3333333333333)]
[(-1e+20, 122.0), (-1e+20, 78.0), (-503.9999999999991, 183.0)]
[(122.0, 1e+20), (78.0, 1e+20), (0.0, 1e+20)]
[(-1e+20, 122.0, 122.0, 1e+20), (-1e+20, 78.0, 78.0, 1e+20), (-503.9999999999991, 183.0, 0.0, 1e+20)]
[(174.0, 207.0), (1440.0, 1800.0), (2712.0, 1e+20)]
The dual solution is:
Dual values of the constraints:
None: 200.00
None: 16.67
None: 0.00
Reduced costs of the variables:
x_0: 0.00
x_1: 0.00
x_2: -13.33
